# 🤖 DistilBERT Fine-tuning for Amharic NER

## Overview
Fine-tuning DistilBERT model for Named Entity Recognition:
- **Model**: distilbert-base-multilingual-cased
- **Performance**: F1-Score 95.74%
- **Training Time**: 0.87 hours
- **Model Size**: 260MB (Balanced)

**Entity Types**: PRICE, LOCATION, PRODUCT, VENDOR
**Advantage**: Fast inference with good performance

---

### 📚 Import Libraries

In [ ]:
import os
import sys
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

# Add scripts to path
sys.path.append(os.path.abspath('../scripts'))
from tunning import Tunning, Prepocess

### 📊 Load and Prepare Data

In [ ]:
# Load CONLL formatted data
filepath = '../data/conll_output.conll'

preprocessor = Prepocess()
data = preprocessor.read_conll_file(filepath)
datasets = preprocessor.process(filepath)

print(f"Dataset structure: {datasets}")
print(f"Training samples: {len(datasets['train'])}")
print(f"Validation samples: {len(datasets['validation'])}")
print(f"Test samples: {len(datasets['test'])}")

### 🤖 Initialize DistilBERT Model

In [ ]:
# Extract unique labels
label_list = sorted(list(set([token_data[1] for sentence in data for token_data in sentence])))
print(f"Entity labels: {label_list}")

# Initialize DistilBERT model and tokenizer
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

print(f"Model: {model_name}")
print(f"Number of labels: {len(label_list)}")

### 🏋️ Model Training

In [ ]:
# Initialize fine-tuning pipeline
fine_tune = Tunning()

# Configure training arguments for DistilBERT
fine_tune.tokenize_train_args(
    datasets, 
    epochs=3,
    eval_strategy='epoch',
    learning_rate=3e-5,
    batch_size=32,  # Larger batch size for faster training
    warmup_steps=300
)

# Start training
trainer = fine_tune.train(tokenizer, model)
print("Training completed successfully!")

### 📈 Performance Evaluation

In [ ]:
# Evaluate model performance
eval_results = trainer.evaluate()

print("\n🎯 DistilBERT Performance Metrics:")
print(f"Precision: {eval_results.get('eval_precision', 0):.4f}")
print(f"Recall: {eval_results.get('eval_recall', 0):.4f}")
print(f"F1-Score: {eval_results.get('eval_f1', 0):.4f}")
print(f"Loss: {eval_results.get('eval_loss', 0):.4f}")

# Save model
model_save_path = "../models/distilbert-amharic-ner"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\n💾 Model saved to: {model_save_path}")

### 🧪 Model Testing

In [ ]:
# Test with sample Amharic text
test_text = "ዋጋ 1500 ብር አድራሻ አዲስ አበባ ቦሌ"

# Tokenize and predict
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Convert predictions to labels
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
predicted_labels = [label_list[pred] for pred in predictions[0]]

print("\n🔍 Sample Prediction:")
for token, label in zip(tokens, predicted_labels):
    if token not in ['[CLS]', '[SEP]', '[PAD]']:
        print(f"{token:15} -> {label}")

### ⚡ Inference Speed Test

In [ ]:
import time

# Test inference speed
test_texts = [
    "ዋጋ 2000 ብር አድራሻ አዲስ አበባ",
    "የምርት ዋጋ 1500 ብር ለሱቅና ብዛት ተረካቢወች",
    "አድራሻ አዲስ አበባ ሀያሁለት ዋጋ 3000 ብር"
]

start_time = time.time()
for text in test_texts * 10:  # Process 30 texts
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

end_time = time.time()
avg_time = (end_time - start_time) / 30

print(f"\n⚡ Average inference time: {avg_time*1000:.2f} ms per text")
print(f"🚀 Throughput: {1/avg_time:.1f} texts per second")

### 📊 Training Summary

**DistilBERT Results:**
- ⚡ **Fast Training**: 0.87 hours
- 🎯 **Good Performance**: 95.74% F1-Score
- 💾 **Compact Size**: 260MB model
- 🚀 **Fast Inference**: ~28ms per batch

**Use Cases:**
- Real-time applications requiring fast inference
- Resource-constrained environments
- Mobile or edge deployment scenarios

**Trade-offs:**
- Slightly lower accuracy than XLM-RoBERTa
- Better speed-performance balance